In [ ]:
# ! pip install qiskit-finance    # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install -U kaleido        # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install jinja2            # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install pylatexenc

# # se, no entanto, estiver rodando localmente é recomendado que seja instalado o python 3.10.11 e instaladas as bibliotecas no requirements.txt
# # !pip install -r requirements.txt

In [ ]:
from qiskit.circuit.library import PauliTwoDesign, TwoLocal, EfficientSU2, RealAmplitudes
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NFT
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, SamplingVQE
from qiskit_aer.primitives import Sampler
from qiskit.quantum_info import Pauli

from IPython.display import clear_output

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import numpy as np
import pickle
import time

# Define Variaveis

In [ ]:
ambiente_backend = 'local'
sampler = Sampler()

seed = 123
fator_risco = 0.5
records_tempo = []

quantidade_epocas_QAOA = 30
quantidade_epocas_VQE = 30

In [ ]:
mostrar_figuras = True
salvar_figuras = False

width_padrao_display = 1400
height_padrao_display = 700

width_padrao_salvar = 1400
height_padrao_salvar = 700

width_padrao_display_junto = 3200
height_padrao_display_junto = 700

width_padrao_salvar_junto = 3200
height_padrao_salvar_junto = 700

tamanho_padrao_fonte = dict(size=35)
tamanho_padrao_margem = dict(l=90, r=90, t=90, b=90)

In [ ]:
global lista_parametros_epoca_algoritmo_quantico
global lista_parametros_iteracao_otimizador
global otimizador_atual
global ansatz_atual
global mixer_atual

def guardar_parametros_iteracao_otimizador_QAOA(parametros):
    lista_parametros_iteracao_otimizador.append((otimizador_atual, mixer_atual, parametros))

def guardar_parametros_iteracao_otimizador_VQE(parametros):
    lista_parametros_iteracao_otimizador.append((otimizador_atual, ansatz_atual, parametros))

otimizadores_QAOA = [COBYLA(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            L_BFGS_B(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            SLSQP(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            NFT(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA)]

otimizadores_VQE = [COBYLA(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE), 
            L_BFGS_B(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE), 
            SLSQP(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE),
            NFT(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE)]


# 4 Ativos

Define variaveis para a quantidade de ativos

In [ ]:
# lista_ativos = ['PETR4','VALE3','PRIO3','PETR3','GGBR4','JBSS3','VBBR3','BRFS3','CSAN3','KLBN4']
lista_ativos = ['CSAN3', 'ITUB4', 'VALE3', 'KLBN4']

# num_portfolios = 3417187 # 10 ativos
num_portfolios = 29192 # 4 ativos

num_ativos = len(lista_ativos)
limite_ativos_comprar = 2
num_qubits = num_ativos

restricao_cardinalidade = False # se aplica apenas ao clássico, por default já temos no quantico

In [ ]:
df_dados_fechamento = pd.read_csv('fechamentos.csv')
df_dados_fechamento = df_dados_fechamento.rename({'ticker':'ativo','fechamento':'fechamento_ajustado'}, axis=1)
df_dados_fechamento = df_dados_fechamento.convert_dtypes()

print(df_dados_fechamento.iloc[np.r_[0:4, -4:0]].to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(df_dados_fechamento)

pivoted_df = df_dados_fechamento.pivot(index='data', columns='ativo', values='fechamento_ajustado')
pivoted_df.index = pd.to_datetime(pivoted_df.index)
pivoted_df = pivoted_df.reindex(sorted(pivoted_df.columns), axis=1)

print(pivoted_df[['PETR4','VALE3','PRIO3','PETR3','GGBR4','JBSS3','VBBR3','BRFS3','CSAN3','KLBN4']].iloc[np.r_[0:4, -4:0]].to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(pivoted_df)

In [ ]:
data_inicio = '2016-01-01'
data_fim = '2023-01-01' 

df_fechamentos = pivoted_df[data_inicio:data_fim][lista_ativos]
df_fechamentos = df_fechamentos.rename_axis('Date')

print(df_fechamentos.iloc[np.r_[0:4, -4:0]].to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_fechamentos

In [ ]:
# Plotando os precos de fechamento
fig = px.line(df_fechamentos, x=df_fechamentos.index, y=lista_ativos, labels={'value': 'Preço de Fechamento', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Preço de Fechamento', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_preços_fechamento.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
fig = px.line(np.log(df_fechamentos + 1), x=df_fechamentos.index, y=lista_ativos, labels={'value': 'Log do preço de Fechamento', 'Date': 'Data'}) # "+1" evita logaritmando negativo para ações cotadas abaixo de "R$ 1"

fig.update_layout(xaxis_title='Data', yaxis_title='Log do preço de Fechamento', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_preços_fechamento_log.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Variação percentual diária ou retorno diário
retorno = df_fechamentos.pct_change()
linhas_nan = retorno[retorno.isna().any(axis=1)]
retorno = retorno.drop(linhas_nan.index)
retorno_porcento = retorno * 100

print(retorno_porcento.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
retorno_porcento

In [ ]:
# Plot do retorno diário dos ativos
fig = px.line(retorno_porcento, x=retorno_porcento.index, y=lista_ativos, labels={'value': 'Retorno diário', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Retorno diário (%)', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display,
                  font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura =  f"figuras_dados/{num_ativos}_retorno.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
log_retorno = np.log(df_fechamentos/df_fechamentos.shift())
linhas_nan = log_retorno[log_retorno.isna().any(axis=1)]
log_retorno = log_retorno.drop(linhas_nan.index)

print(log_retorno.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
log_retorno

In [ ]:
# Plot do retorno logarítmico diário dos ativos
fig = px.line(log_retorno[lista_ativos], x=log_retorno.index, y=lista_ativos,labels={'value': 'Log retorno diário', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Log retorno diário', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_retorno_log.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Cálculo da variança no periodo inteiro com '.var'
varianca_periodo = retorno.var()
varianca_periodo

In [ ]:
# Volatilidade periodo
volatilidade_periodo = np.sqrt(varianca_periodo * len(varianca_periodo))
volatilidade_periodo

In [ ]:
# Plot da volatilidade anual (variança anual)
fig = px.bar(volatilidade_periodo, labels={'value': 'Variança'})
fig.update_layout(xaxis_title='Ativo', yaxis_title='Variança', title='')
fig.update_layout(showlegend=False, width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_varianca.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Matriz covariante
matrixz_cov = df_fechamentos.pct_change().cov()
matrixz_cov = matrixz_cov.dropna()
matrixz_cov_numpy = matrixz_cov.to_numpy()

print(matrixz_cov.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
matrixz_cov

In [ ]:
# Plot da matriz covariante
fig = go.Figure(data=go.Heatmap(z=matrixz_cov.values, x=matrixz_cov.columns, y=matrixz_cov.index))
fig.update_layout(title='Matriz de Covariância', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_covariancia.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Matriz de correlação
matriz_corr = df_fechamentos.pct_change().corr()
matriz_corr = matriz_corr.dropna()

print(matriz_corr.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
matriz_corr

In [ ]:
# Plot da Matriz de correlação
fig = go.Figure(data=go.Heatmap(z=matriz_corr.values, x=matriz_corr.columns, y=matriz_corr.index))
fig.update_layout(title='Matriz de Correlação', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_correlacao.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Exemplo de geração de pesos aleatórios
pesos_exemplo = np.random.random(num_ativos)
pesos_exemplo = pesos_exemplo/np.sum(pesos_exemplo)

print('ativos: ', lista_ativos)
print('pesos : ', pesos_exemplo)

In [ ]:
# Retorno anual médio
retorno_esperado = df_fechamentos.resample('Y').last().pct_change().mean()
retorno_esperado_numpy = retorno_esperado.to_numpy()

retorno_esperado

In [ ]:
# Volatilidade (desvio padrão anualizado)
# A volatilidade é dada pelo desvio padrão anual. Multiplicamos a volatilidade diária por 252 (quantidade de dias de negociação / ano).
desvio_padrao_anualizado = df_fechamentos.pct_change().std().apply(lambda x: x*np.sqrt(252))
desvio_padrao_anualizado

In [ ]:
# Criando um DataFrame com retornos e volatilidade dos portfólios (apenas para display)
df_caracteristicas_ativos = pd.concat([retorno_esperado, desvio_padrao_anualizado], axis=1)
df_caracteristicas_ativos.columns = ['Retorno', 'Risco']

print(df_caracteristicas_ativos.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
df_caracteristicas_ativos

In [ ]:
# Monte Carlo

lista_volatilidades = []
lista_retornos = []
lista_pesos = []

tempo_inicio = time.time()

for portfolio in range(num_portfolios):

    if (restricao_cardinalidade):
        escolha_ativos_portfolio = np.random.random(num_ativos)
        pesos_ativos = np.random.dirichlet(np.ones(limite_ativos_comprar))
        pesos_ativos *= 100

        indices_ativos_escolhidos = np.argsort(escolha_ativos_portfolio)[-limite_ativos_comprar:]

        escolha_ativos_portfolio[:] = 0
        escolha_ativos_portfolio[indices_ativos_escolhidos] = pesos_ativos

        pesos_portfolio = escolha_ativos_portfolio/np.sum(escolha_ativos_portfolio)

    else:
        pesos_portfolio = np.random.random(num_ativos)
        pesos_portfolio = pesos_portfolio/np.sum(pesos_portfolio)
    
    lista_pesos.append(pesos_portfolio)
    retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
    lista_retornos.append(retornos_portfolio)

    variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
    desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
    desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

    lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

tempo_fim = time.time()
tempo_elapsado = str(round(tempo_fim - tempo_inicio,2)).replace('.',',')
record_tempo_monte_carlo = {num_ativos: tempo_elapsado}
records_tempo.append(record_tempo_monte_carlo)

dados_monte_carlo = {'Retorno': lista_retornos, 'Risco': lista_volatilidades}

for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
    dados_monte_carlo['peso '+ ticker] = [w[contador] for w in lista_pesos]

df_portfolios  = pd.DataFrame(dados_monte_carlo)

print(df_portfolios.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
df_portfolios.head(n = 10) # Visualizar os n primeiros portfólio dos portfolios criados com pesos aleatórios

In [ ]:
# Plot da Fronteira Eficiente
fig = px.scatter(df_portfolios, x='Risco', y='Retorno')
fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_pontos_fronteira.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
portfolio_volatilidade_minima = df_portfolios.iloc[df_portfolios['Risco'].idxmin()] # idxmin() retorna o valor mínimo na coluna especificada
portfolio_volatilidade_minima

In [ ]:
portfolio_retorno_maximo = df_portfolios.iloc[df_portfolios['Retorno'].idxmax()] # idxmax() retorna o valor máximo na coluna especificada
portfolio_retorno_maximo

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: Poupança, acumulado 2021 ~ 2022 ~7,8997%

retorno_livre_risco_poupanca = 0.0790
portfolio_otimizado_poupanca = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_poupanca)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_poupanca

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: CDI/SELIC, acumulado 2022 ~12,3910%

retorno_livre_risco_cdi = 0.1239
portfolio_otimizado_cdi = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_cdi)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_cdi

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: FGTS (TR+3%), acumulado 2022 ~1,6314%

retorno_livre_risco_fgts = 0.0163
portfolio_otimizado_fgts = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_fgts)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_fgts

In [ ]:
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
    go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
    go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
]

fig = go.Figure(data=[*objetos_scatter])
fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_pontos_portfolios_fronteira.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Fit simples para extrair a curva de fronteira eficiente
minimo_random_shape = 0.02
maximo_random_shape = 0.13

minimo_random_min_vol = 0.15
maximo_random_min_vol = 0.3

valor_aleatorio_1 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_1 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_1]['Risco']).idxmin()]

valor_aleatorio_2 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_2 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_2]['Risco']).idxmin()]

valor_aleatorio_3 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_3 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_3]['Risco']).idxmin()]

valor_aleatorio_4 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_4 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_4)/df_portfolios['Risco']).idxmax()]

valor_aleatorio_5 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_5 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_5)/df_portfolios['Risco']).idxmax()]

valor_aleatorio_6 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_6 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_6)/df_portfolios['Risco']).idxmax()]

# Selecionando alguns pontos da fronteira
retornos_selecionados = np.array([portfolio_volatilidade_minima[1], ponto_aleatorio_fronteira_1[1], ponto_aleatorio_fronteira_2[1], ponto_aleatorio_fronteira_3[1], ponto_aleatorio_fronteira_4[1], ponto_aleatorio_fronteira_5[1], ponto_aleatorio_fronteira_6[1]])
riscos_selecionados = np.array([portfolio_volatilidade_minima[0], ponto_aleatorio_fronteira_1[0], ponto_aleatorio_fronteira_2[0], ponto_aleatorio_fronteira_3[0], ponto_aleatorio_fronteira_4[0], ponto_aleatorio_fronteira_5[0], ponto_aleatorio_fronteira_6[0]])

# Obs: Vamos fazer o fit polinomial com grau "g" de retornos em função de riscos, pois o contrário não é função
grau = 2
coeficientes = np.polyfit(riscos_selecionados, retornos_selecionados, grau)  # Retornos em função de riscos
modelo_ajustado = np.poly1d(coeficientes)

# Criando a curva de novos retornos e riscos após o fit e calibrando os limites
risco_novo = np.linspace(riscos_selecionados[0] - 0.05, riscos_selecionados[-1] + 0.02, 50)
retorno_novo = modelo_ajustado(risco_novo)

In [ ]:
# Plotando novamente com a fronteira eficiente explícita
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=retorno_novo, y=risco_novo, mode='lines', line=dict(color='black', width=3), name='Fronteira Eficiente'),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[ponto_aleatorio_fronteira_1.iloc[1]], y=[ponto_aleatorio_fronteira_1.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='diamond'),text=['ponto_aleatorio_fronteira_1'], name='ponto_aleatorio_fronteira_1'),
    go.Scatter(x=[ponto_aleatorio_fronteira_2.iloc[1]], y=[ponto_aleatorio_fronteira_2.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_2'], name='ponto_aleatorio_fronteira_2'),
    go.Scatter(x=[ponto_aleatorio_fronteira_3.iloc[1]], y=[ponto_aleatorio_fronteira_3.iloc[0]], mode='markers', marker=dict(color='brown', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_3'], name='ponto_aleatorio_fronteira_3'),
    go.Scatter(x=[ponto_aleatorio_fronteira_4.iloc[1]], y=[ponto_aleatorio_fronteira_4.iloc[0]], mode='markers', marker=dict(color='black', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_4'], name='ponto_aleatorio_fronteira_4'),
    go.Scatter(x=[ponto_aleatorio_fronteira_5.iloc[1]], y=[ponto_aleatorio_fronteira_5.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='diamond'),  text=['ponto_aleatorio_fronteira_5'], name='ponto_aleatorio_fronteira_5'),
    go.Scatter(x=[ponto_aleatorio_fronteira_6.iloc[1]], y=[ponto_aleatorio_fronteira_6.iloc[0]], mode='markers', marker=dict(color='cyan', size=12, symbol='diamond'),  text=['ponto_aleatorio_fronteira_6'], name='ponto_aleatorio_fronteira_6'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_pontos_aleatorios.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Plotando novamente com a fronteira eficiente explícita
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=retorno_novo, y=risco_novo, mode='lines', line=dict(color='black', width=3), name='Fronteira Eficiente'),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
    go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
    go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")


if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

# Quantum

In [ ]:
# bounds = [(0,100) for i in range(num_qubits)]
# bounds

In [ ]:
otimizacao_portfolio = PortfolioOptimization(
    expected_returns = retorno_esperado_numpy, covariances = matrixz_cov_numpy,
    risk_factor = fator_risco, budget = limite_ativos_comprar
)

programa_quadratico = otimizacao_portfolio.to_quadratic_program()
programa_quadratico_4 = programa_quadratico
programa_quadratico

### VQE

In [ ]:
ansatzes = [PauliTwoDesign(num_qubits, reps=1),
            EfficientSU2(num_qubits),
            RealAmplitudes(num_qubits),
            TwoLocal(num_qubits, "ry", "cz", reps=1, entanglement="full")]

lista_df_resultados_vqe = []
lista_parametros_iteracao_otimizador = []
lista_parametros_epoca_algoritmo_quantico = []

for indice_otimizador, otimizador in enumerate(otimizadores_VQE):

    otimizador_atual = type(otimizador).__name__

    lista_ansatz = []
    lista_otimizador = []

    lista_parametros = []
    lista_valor_energia = []
    lista_avaliacao_funcao_custo = []

    lista_epocas = []
    lista_composicao_otima = []
    lista_status_otimizacao = []
    lista_amostras_otimizacao = []
    lista_valor_otimo_encontrado = []
    lista_tempo_elapsado_otimizador = []
    lista_probabilidade_valor_otimo = []
    lista_quantidade_avaliacoes_total = []

    for indice_ansatz, ansatz in enumerate(ansatzes):
        ansatz_atual = ansatz.name

        print(f'\rOptimizer: {otimizador_atual}\t' + f'\tAnsatz: {ansatz_atual} \t\t\t\t\t\t', end='')

        melhores_parametros = None

        for epoca in range(quantidade_epocas_VQE):

            lista_avaliacoes_aux = []
            lista_parametros_aux = []
            lista_energia_aux = []

            def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
                lista_avaliacoes_aux.append(eval_count)
                lista_parametros_aux.append(parameters)
                lista_energia_aux.append(mean)
                clear_output(wait=False)

            # vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=otimizador, callback=guardar_parametros_chamada_custo)
            vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=otimizador, initial_point=melhores_parametros, callback=guardar_parametros_chamada_custo)
            vqe.random_seed = seed
            vqe_eigen = MinimumEigenOptimizer(vqe)
            resultado = vqe_eigen.solve(programa_quadratico)

            indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
            melhores_parametros = lista_parametros_aux[indice_menor_energia]
            # guardar_resultados_epoca_algoritmo_quantico_VQE(resultado, epoca)

            objeto_resultado = resultado.min_eigen_solver_result

            composicao_otima = resultado.x
            status_otimizacao = resultado.status
            amostras_otimizacao = resultado.samples
            valor_otimo_encontrado = objeto_resultado.optimal_value
            tempo_elapsado_otimizador = objeto_resultado.optimizer_time
            probabilidade_valor_otimo = objeto_resultado.best_measurement['probability']
            quantidade_avaliacoes_total = objeto_resultado.cost_function_evals

            lista_avaliacao_funcao_custo += lista_avaliacoes_aux
            lista_parametros += lista_parametros_aux
            lista_valor_energia += lista_energia_aux

            tamanho_listas = len(lista_energia_aux)

            lista_ansatz += [ansatz_atual for i in range(tamanho_listas)]
            lista_otimizador += [otimizador_atual for i in range(tamanho_listas)]

            lista_epocas += [epoca + 1 for i in range(tamanho_listas)]
            lista_composicao_otima += [composicao_otima for i in range(tamanho_listas)]
            lista_status_otimizacao += [status_otimizacao for i in range(tamanho_listas)]
            lista_amostras_otimizacao += [amostras_otimizacao for i in range(tamanho_listas)]
            lista_valor_otimo_encontrado += [valor_otimo_encontrado for i in range(tamanho_listas)]
            lista_tempo_elapsado_otimizador += [tempo_elapsado_otimizador for i in range(tamanho_listas)]
            lista_probabilidade_valor_otimo += [probabilidade_valor_otimo for i in range(tamanho_listas)]
            lista_quantidade_avaliacoes_total += [quantidade_avaliacoes_total for i in range(tamanho_listas)]
        
    dados_df_otimizador = {
        'parametros': lista_parametros,
        'epoca': lista_epocas,
        'avaliacao_funcao_custo': lista_avaliacao_funcao_custo,
        'valor_energia': lista_valor_energia,
        'ansatz': lista_ansatz,
        'otimizador': lista_otimizador,
        'composicao_otima': lista_composicao_otima,
        'status_otimizacao': lista_status_otimizacao,
        'amostras_otimizacao': lista_amostras_otimizacao,
        'quantidade_avaliacoes_total': lista_quantidade_avaliacoes_total,
        'tempo_elapsado_otimizador': lista_tempo_elapsado_otimizador,
        'valor_otimo_encontrado': lista_valor_otimo_encontrado,
        'probabilidade_valor_otimo': lista_probabilidade_valor_otimo,
    }

    df_otimizador = pd.DataFrame(dados_df_otimizador)

    lista_df_resultados_vqe.append(df_otimizador)

    array_avaliacao_funcao_custo_filtrado = np.empty([len(otimizadores_VQE)], dtype=object)
    array_valor_energia_filtrado = np.empty([len(otimizadores_VQE)], dtype=object)

    aux_contador_otimizador = 0

df_vqe = pd.concat(lista_df_resultados_vqe)
df_vqe = df_vqe.reset_index(drop=True)
df_vqe['nova_epoca'] = df_vqe['epoca'].diff().fillna(0).astype(bool)

clear_output(wait=True)
print(df_vqe.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_vqe

In [ ]:
def checa_final_iteracao_otimizador(row):
    for otimizador, ansatz, parametros in lista_parametros_iteracao_otimizador:
        if row['otimizador'] == otimizador and row['ansatz'] == ansatz and tuple(row['parametros']) == tuple(parametros):
            return True
        elif row['otimizador'] == otimizador and row['ansatz'] == ansatz and row['avaliacao_funcao_custo'] == 1: # Lógica extra para a primeira chamada, pois o callback só da o retorno e nao o input.
            return True
    return False

df_vqe['bool_fim_iteracao_otimizador'] = df_vqe.apply(checa_final_iteracao_otimizador, axis=1)
df_vqe['iteracao_otimizador'] = np.nan

# cria indice de iteracoes otimizador
for _, df_agrupado in df_vqe.loc[df_vqe.bool_fim_iteracao_otimizador == True].groupby(['ansatz', 'otimizador'], as_index=False):
    df_vqe.loc[df_agrupado.index, 'iteracao_otimizador'] = df_agrupado.reset_index().index + 1

df_vqe

In [ ]:
with open(f'pickles/vqe_{num_ativos}_{ambiente_backend}.pickle', 'wb') as file:
    pickle.dump(df_vqe, file)

In [ ]:
# for otimizador in df_vqe['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe[df_vqe['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='avaliacao_funcao_custo')

#         x_data = df_ansatz['avaliacao_funcao_custo']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display,
#         font=tamanho_padrao_fonte,
#         margin=tamanho_padrao_margem)

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real = df_vqe_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_vqe_real, x='avaliacao_funcao_custo', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
df_vqe_iteracao_otimizador = df_vqe.loc[df_vqe['bool_fim_iteracao_otimizador'] == True]

for otimizador in df_vqe_iteracao_otimizador['otimizador'].unique():
    fig = go.Figure()

    df_otimizador = df_vqe_iteracao_otimizador[df_vqe_iteracao_otimizador['otimizador'] == otimizador]

    for ansatz in df_otimizador['ansatz'].unique():
        df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
        df_ansatz = df_ansatz.sort_values(by='iteracao_otimizador')

        x_data = df_ansatz['iteracao_otimizador']
        y_data = df_ansatz['valor_energia']

        y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_real,
            mode='lines',
            name=ansatz
        ))

    fig.update_layout(
        xaxis_title='Iteração do otimizador',
        yaxis_title='Energia',
        title='',
        legend_title='Ansatzes',
        width=width_padrao_display,
        height=height_padrao_display, 
        font=tamanho_padrao_fonte, 
        margin=tamanho_padrao_margem
    )

    if salvar_figuras:
        nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_vqe.png"
        fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

    if mostrar_figuras:
        fig.show()

In [ ]:
df_vqe_real = df_vqe.copy()
df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_vqe_real_iteracao_otimizador = df_vqe_real.loc[df_vqe_real['bool_fim_iteracao_otimizador'] == True]
df_vqe_real_iteracao_otimizador = df_vqe_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

fig = px.line(df_vqe_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='ansatz',
              facet_col='otimizador', facet_col_wrap=df_vqe_real_iteracao_otimizador['otimizador'].nunique(),
              labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

fig.add_annotation(
    text='Iteração do otimizador',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.178,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_otimizador_juntos_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_vqe_epoca = df_vqe.loc[df_vqe['nova_epoca'] == True]

for otimizador in df_vqe_epoca['otimizador'].unique():
    df_otimizador = df_vqe_epoca[df_vqe_epoca['otimizador'] == otimizador]

    fig = go.Figure()

    for ansatz in df_otimizador['ansatz'].unique():
        df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
        df_ansatz = df_ansatz.sort_values(by='epoca')

        x_data = df_ansatz['epoca']
        y_data = df_ansatz['valor_energia']

        # Handling complex numbers by extracting real and imaginary parts
        y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_real,
            mode='lines',
            name=ansatz
        ))

    fig.update_layout(
        xaxis_title='Epoca',
        yaxis_title='Energia',
        title='',
        legend_title='ansatzs',
        width=width_padrao_display,
        height=height_padrao_display, 
        font=tamanho_padrao_fonte, 
        margin=tamanho_padrao_margem
    )

    if salvar_figuras:
        nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_vqe.png"
        fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

    if mostrar_figuras:
        fig.show()

In [ ]:
df_vqe_real = df_vqe.copy()
df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_vqe_real_epoca = df_vqe_real.loc[df_vqe_real['nova_epoca'] == True]
df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by='epoca')

fig = px.line(df_vqe_real_epoca, x='epoca', y='valor_energia', color='ansatz',
              facet_col='otimizador', facet_col_wrap=df_vqe_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='valor_otimo_encontrado', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='probabilidade_valor_otimo', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='tempo_elapsado_otimizador', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_tempo_elapsado_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='quantidade_avaliacoes_total', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='ansatz', y='iteracao_otimizador', color='ansatz', facet_col='otimizador',
#              title='', 
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_otimizador_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### QAOA

In [ ]:
# Function to create a ring mixer
def create_ring_mixer(num_qubits):
    ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        ring_mixer *= Pauli(''.join(pauli_list))
    return ring_mixer

# Function to create a parity ring mixer
def create_parity_ring_mixer(num_qubits):
    parity_ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        parity_ring_mixer *= Pauli(''.join(pauli_list))
    pauli_list[-1] = 'X'  # Apply X gate on the last qubit
    parity_ring_mixer *= Pauli(''.join(pauli_list))
    return parity_ring_mixer

# Function to create a full mixer
def create_full_mixer(num_qubits):
    return Pauli('X'*num_qubits)

ring_mixer = create_ring_mixer(num_qubits)
parity_ring_mixer = create_parity_ring_mixer(num_qubits)
full_mixer = create_full_mixer(num_qubits)

In [ ]:
mixers = [('mixer_default', None),
          ('ring_mixer', ring_mixer),
          ('parity_ring_mixer', parity_ring_mixer),
          ('full_mixer', full_mixer)]

# print('\rOtimizacao completa\t\t\t\t\t\t\t')

lista_df_resultados_qaoa = []
lista_parametros_iteracao_otimizador = []
lista_parametros_epoca_algoritmo_quantico = []

for indice_otimizador, otimizador in enumerate(otimizadores_QAOA):

    otimizador_atual = type(otimizador).__name__

    lista_mixer = []
    lista_otimizador = []

    lista_parametros = []
    lista_valor_energia = []
    lista_avaliacao_funcao_custo = []

    lista_epocas = []
    lista_composicao_otima = []
    lista_status_otimizacao = []
    lista_amostras_otimizacao = []
    lista_valor_otimo_encontrado = []
    lista_tempo_elapsado_otimizador = []
    lista_probabilidade_valor_otimo = []
    lista_quantidade_avaliacoes_total = []

    for indice_mixer, tupla_mixer in enumerate(mixers):
        
        mixer = tupla_mixer[1]
        mixer_atual = tupla_mixer[0]

        print(f'\rOptimizer: {otimizador_atual}\t' + f'\tmixer: {mixer_atual} \t\t\t\t\t\t', end='')

        melhores_parametros = None

        for epoca in range(quantidade_epocas_QAOA):

            lista_avaliacoes_aux = []
            lista_parametros_aux = []
            lista_energia_aux = []

            def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
                lista_avaliacoes_aux.append(eval_count)
                lista_parametros_aux.append(parameters)
                lista_energia_aux.append(mean)
                clear_output(wait=False)

            # qaoa = QAOA(sampler=sampler, optimizer=otimizador, reps=3, mixer=mixer, callback=guardar_parametros_chamada_custo)
            qaoa = QAOA(sampler=sampler, optimizer=otimizador, reps=1, mixer=mixer, initial_point=melhores_parametros, callback=guardar_parametros_chamada_custo)
            qaoa_eigen = MinimumEigenOptimizer(qaoa)
            resultado = qaoa_eigen.solve(programa_quadratico)

            indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
            melhores_parametros = lista_parametros_aux[indice_menor_energia]

            objeto_resultado = resultado.min_eigen_solver_result

            composicao_otima = resultado.x
            status_otimizacao = resultado.status
            amostras_otimizacao = resultado.samples
            valor_otimo_encontrado = objeto_resultado.optimal_value
            tempo_elapsado_otimizador = objeto_resultado.optimizer_time
            probabilidade_valor_otimo = objeto_resultado.best_measurement['probability']
            quantidade_avaliacoes_total = objeto_resultado.cost_function_evals

            lista_avaliacao_funcao_custo += lista_avaliacoes_aux
            lista_parametros += lista_parametros_aux
            lista_valor_energia += lista_energia_aux

            tamanho_listas = len(lista_energia_aux)

            lista_mixer += [mixer_atual for i in range(tamanho_listas)]
            lista_otimizador += [otimizador_atual for i in range(tamanho_listas)]

            lista_epocas += [epoca + 1 for i in range(tamanho_listas)]
            lista_composicao_otima += [composicao_otima for i in range(tamanho_listas)]
            lista_status_otimizacao += [status_otimizacao for i in range(tamanho_listas)]
            lista_amostras_otimizacao += [amostras_otimizacao for i in range(tamanho_listas)]
            lista_valor_otimo_encontrado += [valor_otimo_encontrado for i in range(tamanho_listas)]
            lista_tempo_elapsado_otimizador += [tempo_elapsado_otimizador for i in range(tamanho_listas)]
            lista_probabilidade_valor_otimo += [probabilidade_valor_otimo for i in range(tamanho_listas)]
            lista_quantidade_avaliacoes_total += [quantidade_avaliacoes_total for i in range(tamanho_listas)]
        
    dados_df_otimizador = {
        'parametros': lista_parametros,
        'epoca': lista_epocas,
        'avaliacao_funcao_custo': lista_avaliacao_funcao_custo,
        'valor_energia': lista_valor_energia,
        'mixer': lista_mixer,
        'otimizador': lista_otimizador,
        'composicao_otima': lista_composicao_otima,
        'status_otimizacao': lista_status_otimizacao,
        'amostras_otimizacao': lista_amostras_otimizacao,
        'quantidade_avaliacoes_total': lista_quantidade_avaliacoes_total,
        'tempo_elapsado_otimizador': lista_tempo_elapsado_otimizador,
        'valor_otimo_encontrado': lista_valor_otimo_encontrado,
        'probabilidade_valor_otimo': lista_probabilidade_valor_otimo,
    }

    df_otimizador = pd.DataFrame(dados_df_otimizador)

    lista_df_resultados_qaoa.append(df_otimizador)

    array_avaliacao_funcao_custo_filtrado = np.empty([len(otimizadores_QAOA)], dtype=object)
    array_valor_energia_filtrado = np.empty([len(otimizadores_QAOA)], dtype=object)

    aux_contador_otimizador = 0

df_qaoa = pd.concat(lista_df_resultados_qaoa)
df_qaoa = df_qaoa.reset_index(drop=True)
df_qaoa['nova_epoca'] = df_qaoa['epoca'].diff().fillna(0).astype(bool)

clear_output(wait=True)
print(df_qaoa.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_qaoa

In [ ]:
# def checa_final_iteracao_otimizador(row):
#     for otimizador, mixer, parametros in lista_parametros_iteracao_otimizador:
#         if row['otimizador'] == otimizador and row['mixer'] == mixer and tuple(row['parametros']) == tuple(parametros):
#             return True
#         elif row['otimizador'] == otimizador and row['mixer'] == mixer and row['avaliacao_funcao_custo'] == 1:
#             return True
#     return False

# df_qaoa['bool_fim_iteracao_otimizador'] = df_qaoa.apply(checa_final_iteracao_otimizador, axis=1)
# df_qaoa['iteracao_otimizador'] = np.nan

# # cria indice de iteracoes otimizador
# for _, df_agrupado in df_qaoa.loc[df_qaoa.bool_fim_iteracao_otimizador == True].groupby(['mixer', 'otimizador'], as_index=False):
#     df_qaoa.loc[df_agrupado.index, 'iteracao_otimizador'] = df_agrupado.reset_index().index + 1

# df_qaoa

In [ ]:
with open(f'pickles/qaoa_{num_ativos}_{ambiente_backend}.pickle', 'wb') as file:
    pickle.dump(df_qaoa, file)

In [ ]:
# for otimizador in df_qaoa['otimizador'].unique():
#     df_otimizador = df_qaoa[df_qaoa['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='avaliacao_funcao_custo')
        

#         x_data = df_mixer['avaliacao_funcao_custo']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )


#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real = df_qaoa_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_qaoa_real, x='avaliacao_funcao_custo', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')


# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
# df_qaoa_iteracao_otimizador = df_qaoa.loc[df_qaoa['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_qaoa_iteracao_otimizador['otimizador'].unique():
#     df_otimizador = df_qaoa_iteracao_otimizador[df_qaoa_iteracao_otimizador['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='iteracao_otimizador')

#         x_data = df_mixer['iteracao_otimizador']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real_iteracao_otimizador = df_qaoa_real.loc[df_qaoa_real['bool_fim_iteracao_otimizador'] == True]
# df_qaoa_real_iteracao_otimizador = df_qaoa_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_qaoa_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_otimizador_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
df_qaoa_epoca = df_qaoa.loc[df_qaoa['nova_epoca'] == True]

for otimizador in df_qaoa_epoca['otimizador'].unique():
    df_otimizador = df_qaoa_epoca[df_qaoa_epoca['otimizador'] == otimizador]

    fig = go.Figure()

    for mixer in df_otimizador['mixer'].unique():
        df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
        df_mixer = df_mixer.sort_values(by='epoca')

        x_data = df_mixer['epoca']
        y_data = df_mixer['valor_energia']

        # Handling complex numbers by extracting real and imaginary parts
        y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_real,
            mode='lines',
            name=mixer
        ))

    fig.update_layout(
        xaxis_title='Epoca',
        yaxis_title='Energia',
        title='',
        legend_title='mixers',
        width=width_padrao_display,
        height=height_padrao_display, 
        font=tamanho_padrao_fonte, 
        margin=tamanho_padrao_margem
    )

    if salvar_figuras:
        nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_qaoa.png"
        fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

    if mostrar_figuras:
        fig.show()

In [ ]:
df_qaoa_real = df_qaoa.copy()
df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_qaoa_real_epoca = df_qaoa_real.loc[df_qaoa_real['nova_epoca'] == True]
df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by='epoca')

fig = px.line(df_qaoa_real_epoca, x='epoca', y='valor_energia', color='mixer',
              facet_col='otimizador', facet_col_wrap=df_qaoa_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='valor_otimo_encontrado', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_valor_otimo_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='probabilidade_valor_otimo', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.178,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_qaoa.png" 
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='tempo_elapsado_otimizador', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_tempo_elapsado_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='quantidade_avaliacoes_total', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='mixer', y='iteracao_otimizador', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_otimizador_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### Plots finais nas fronteiras eficientes

In [ ]:
lista_pesos_portfolio_quantico = []
composicao_portfolio_quantico = df_qaoa.composicao_otima.iloc[0]

pesos_portfolio_quantico = composicao_portfolio_quantico/np.sum(composicao_portfolio_quantico)
lista_pesos_portfolio_quantico.append(pesos_portfolio_quantico)

retorno_esperado_portfolio_quantico = np.dot(pesos_portfolio_quantico, retorno_esperado)

variancia_portfolio_quantico = matrixz_cov.mul(pesos_portfolio_quantico, axis=0).mul(pesos_portfolio_quantico, axis=1).sum().sum()
desvio_padrao_diario_portfolio_quantico = np.sqrt(variancia_portfolio_quantico)
desvio_padrao_anualizado_portfolio_quantico = desvio_padrao_diario_portfolio_quantico*np.sqrt(252)

dados_portfolio_quantico = {'Retorno': retorno_esperado_portfolio_quantico, 'Risco': desvio_padrao_anualizado_portfolio_quantico}

for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
    dados_portfolio_quantico['peso '+ ticker] = [w[contador] for w in lista_pesos_portfolio_quantico]

df_portfolio_quantico  = pd.DataFrame(dados_portfolio_quantico)

print('Carteira otimizada com algoritmos quânticos:')
print(df_portfolio_quantico.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(df_portfolio_quantico)

sem_restricao_lista_volatilidades = []
sem_restricao_lista_retornos = []
sem_restricao_lista_pesos = []
sem_restricao_dados_monte_carlo = {}

for portfolio in range(num_portfolios*4):

    pesos_portfolio = np.random.random(num_ativos)
    pesos_portfolio = pesos_portfolio/np.sum(pesos_portfolio)
    
    sem_restricao_lista_pesos.append(pesos_portfolio)
    retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
    sem_restricao_lista_retornos.append(retornos_portfolio)

    variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
    desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
    desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

    sem_restricao_lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

sem_restricao_dados_monte_carlo = {'Retorno': sem_restricao_lista_retornos, 'Risco': sem_restricao_lista_volatilidades}

for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
    sem_restricao_dados_monte_carlo['peso '+ ticker] = [w[contador] for w in sem_restricao_lista_pesos]

df_portfolios_sem_restricao = pd.DataFrame(sem_restricao_dados_monte_carlo)

print('Carteiras sem restrição:')
print(df_portfolios_sem_restricao.head(5).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(df_portfolios_sem_restricao.head(n = 3))

cardinalidade_lista_volatilidades = []
cardinalidade_lista_retornos = []
cardinalidade_lista_pesos = []
cardinalidade_dados_monte_carlo = {}

for portfolio in range(num_portfolios*4):
    escolha_ativos_portfolio = np.random.random(num_ativos)
    pesos_ativos = np.random.dirichlet(np.ones(limite_ativos_comprar))
    pesos_ativos *= 100

    indices_ativos_escolhidos = np.argsort(escolha_ativos_portfolio)[-limite_ativos_comprar:]

    escolha_ativos_portfolio[:] = 0
    escolha_ativos_portfolio[indices_ativos_escolhidos] = pesos_ativos

    pesos_portfolio = escolha_ativos_portfolio/np.sum(escolha_ativos_portfolio)

    cardinalidade_lista_pesos.append(pesos_portfolio)

    retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
    cardinalidade_lista_retornos.append(retornos_portfolio)

    variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
    desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
    desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

    cardinalidade_lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

cardinalidade_dados_monte_carlo = {'Retorno': cardinalidade_lista_retornos, 'Risco': cardinalidade_lista_volatilidades}

for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
    cardinalidade_dados_monte_carlo['peso '+ ticker] = [w[contador] for w in cardinalidade_lista_pesos]

df_portfolios_cardinalidade  = pd.DataFrame(cardinalidade_dados_monte_carlo)

print('Carteiras com restrição de cardinalidade:')
print(df_portfolios_cardinalidade.head(5).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(df_portfolios_cardinalidade.head(n = 3))


objetos_scatter = [
    go.Scatter(x=df_portfolios_sem_restricao['Risco'], y=df_portfolios_sem_restricao['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=df_portfolios_cardinalidade['Risco'], y=df_portfolios_cardinalidade['Retorno'], mode='markers', marker=dict(color='cyan'), text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
    go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
    go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
    go.Scatter(x=[df_portfolio_quantico['Risco'].iloc[0]], y=[df_portfolio_quantico['Retorno'].iloc[0]], mode='markers', marker=dict(color='black', size=20, symbol='x'),text=['Portfólio Otimizado Quântico'], name='Portfólio Otimizado Quântico'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")


if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_quantico_comparativo.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()


objetos_scatter = [
    go.Scatter(x=df_portfolios_sem_restricao['Risco'], y=df_portfolios_sem_restricao['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=df_portfolios_cardinalidade['Risco'], y=df_portfolios_cardinalidade['Retorno'], mode='markers', marker=dict(color='cyan'), text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=[df_portfolio_quantico['Risco'].iloc[0]], y=[df_portfolio_quantico['Retorno'].iloc[0]], mode='markers', marker=dict(color='black', size=20, symbol='x'),text=['Portfólio Otimizado Quântico'], name='Portfólio Otimizado Quântico'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")


if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_quantico.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

# 10 Ativos

Define variaveis para a quantidade de ativos

In [ ]:
lista_ativos = ['PETR4','VALE3','PRIO3','PETR3','GGBR4','JBSS3','VBBR3','BRFS3','CSAN3','KLBN4']
# lista_ativos = ['CSAN3', 'ITUB4', 'VALE3', 'KLBN4']

num_portfolios = 3417187 # 10 ativos
# num_portfolios = 29192 # 4 ativos

num_ativos = len(lista_ativos)
limite_ativos_comprar = 2
num_qubits = num_ativos

restricao_cardinalidade = False # se aplica apenas ao clássico, por default já temos no quantico

In [ ]:
df_dados_fechamento = pd.read_csv('fechamentos.csv')
df_dados_fechamento = df_dados_fechamento.rename({'ticker':'ativo','fechamento':'fechamento_ajustado'}, axis=1)
df_dados_fechamento = df_dados_fechamento.convert_dtypes()

print(df_dados_fechamento.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
display(df_dados_fechamento)

pivoted_df = df_dados_fechamento.pivot(index='data', columns='ativo', values='fechamento_ajustado')
pivoted_df.index = pd.to_datetime(pivoted_df.index)
pivoted_df = pivoted_df.reindex(sorted(pivoted_df.columns), axis=1)

In [ ]:
data_inicio = '2016-01-01'
data_fim = '2023-01-01'

df_fechamentos = pivoted_df[data_inicio:data_fim][lista_ativos]
df_fechamentos = df_fechamentos.rename_axis('Date')

print(df_fechamentos.tail(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_fechamentos

In [ ]:
# Plotando os precos de fechamento
fig = px.line(df_fechamentos, x=df_fechamentos.index, y=lista_ativos, labels={'value': 'Preço de Fechamento', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Preço de Fechamento', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, 
                  font=tamanho_padrao_fonte, margin=tamanho_padrao_margem,
                  legend_title_font=dict(size=25))

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_preços_fechamento.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
fig = px.line(np.log(df_fechamentos + 1), x=df_fechamentos.index, y=lista_ativos, labels={'value': 'Log do preço de Fechamento', 'Date': 'Data'}) # "+1" evita logaritmando negativo para ações cotadas abaixo de "R$ 1"

fig.update_layout(xaxis_title='Data', yaxis_title='Log do preço de Fechamento', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, 
                  font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_preços_fechamento_log.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Variação percentual diária ou retorno diário
retorno = df_fechamentos.pct_change()
linhas_nan = retorno[retorno.isna().any(axis=1)]
retorno = retorno.drop(linhas_nan.index)
retorno_porcento = retorno * 100

print(retorno_porcento.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
retorno_porcento

In [ ]:
# Plot do retorno diário dos ativos
fig = px.line(retorno_porcento, x=retorno_porcento.index, y=lista_ativos, labels={'value': 'Retorno diário', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Retorno diário (%)', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display,
                  font=tamanho_padrao_fonte, margin=tamanho_padrao_margem,
                  legend_title_font=dict(size=25))

if salvar_figuras:
    nome_figura =  f"figuras_dados/{num_ativos}_retorno.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
log_retorno = np.log(df_fechamentos/df_fechamentos.shift())
linhas_nan = log_retorno[log_retorno.isna().any(axis=1)]
log_retorno = log_retorno.drop(linhas_nan.index)

print(log_retorno.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
log_retorno

In [ ]:
# Plot do retorno logarítmico diário dos ativos
fig = px.line(log_retorno[lista_ativos], x=log_retorno.index, y=lista_ativos,labels={'value': 'Log retorno diário', 'Date': 'Data'})

fig.update_layout(xaxis_title='Data', yaxis_title='Log retorno diário', title='',
                  legend_title='Ativo', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_retorno_log.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Cálculo da variança no periodo inteiro com '.var'
varianca_periodo = retorno.var()
varianca_periodo

In [ ]:
# Volatilidade periodo
volatilidade_periodo = np.sqrt(varianca_periodo * len(varianca_periodo))
volatilidade_periodo

In [ ]:
# Plot da volatilidade anual (variança anual)
fig = px.bar(volatilidade_periodo, labels={'value': 'Variança'})
fig.update_layout(xaxis_title='Ativo', yaxis_title='Variança', title='')
fig.update_layout(showlegend=False, width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_varianca.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Matriz covariante
matrixz_cov = df_fechamentos.pct_change().cov()
matrixz_cov = matrixz_cov.dropna()
matrixz_cov_numpy = matrixz_cov.to_numpy()

print(matrixz_cov.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
matrixz_cov

In [ ]:
# Plot da matriz covariante
fig = go.Figure(data=go.Heatmap(z=matrixz_cov.values, x=matrixz_cov.columns, y=matrixz_cov.index))
fig.update_layout(title='Matriz de Covariância', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_covariancia.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Matriz de correlação
matriz_corr = df_fechamentos.pct_change().corr()
matriz_corr = matriz_corr.dropna()

print(matriz_corr.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
matriz_corr

In [ ]:
# Plot da Matriz de correlação
fig = go.Figure(data=go.Heatmap(z=matriz_corr.values, x=matriz_corr.columns, y=matriz_corr.index))
fig.update_layout(title='Matriz de Correlação', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_correlacao.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Exemplo de geração de pesos aleatórios
pesos_exemplo = np.random.random(num_ativos)
pesos_exemplo = pesos_exemplo/np.sum(pesos_exemplo)

print('ativos: ', lista_ativos)
print('pesos : ', pesos_exemplo)

In [ ]:
# Retorno anual médio
retorno_esperado = df_fechamentos.resample('Y').last().pct_change().mean()
retorno_esperado_numpy = retorno_esperado.to_numpy()

retorno_esperado

In [ ]:
# Volatilidade (desvio padrão anualizado)
# A volatilidade é dada pelo desvio padrão anual. Multiplicamos a volatilidade diária por 252 (quantidade de dias de negociação / ano).
desvio_padrao_anualizado = df_fechamentos.pct_change().std().apply(lambda x: x*np.sqrt(252))
desvio_padrao_anualizado

In [ ]:
# Criando um DataFrame com retornos e volatilidade dos portfólios (apenas para display)
df_caracteristicas_ativos = pd.concat([retorno_esperado, desvio_padrao_anualizado], axis=1)
df_caracteristicas_ativos.columns = ['Retorno', 'Risco']

print(df_caracteristicas_ativos.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
df_caracteristicas_ativos

In [ ]:
# Monte Carlo

lista_volatilidades = []
lista_retornos = []
lista_pesos = []

tempo_inicio = time.time()

for portfolio in range(num_portfolios):

    if (restricao_cardinalidade):
        escolha_ativos_portfolio = np.random.random(num_ativos)
        pesos_ativos = np.random.dirichlet(np.ones(limite_ativos_comprar))
        pesos_ativos *= 100

        indices_ativos_escolhidos = np.argsort(escolha_ativos_portfolio)[-limite_ativos_comprar:]

        escolha_ativos_portfolio[:] = 0
        escolha_ativos_portfolio[indices_ativos_escolhidos] = pesos_ativos

        pesos_portfolio = escolha_ativos_portfolio/np.sum(escolha_ativos_portfolio)

    else:
        pesos_portfolio = np.random.random(num_ativos)
        pesos_portfolio = pesos_portfolio/np.sum(pesos_portfolio)
    
    lista_pesos.append(pesos_portfolio)
    retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
    lista_retornos.append(retornos_portfolio)

    variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
    desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
    desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

    lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

tempo_fim = time.time()
tempo_elapsado = str(round(tempo_fim - tempo_inicio,2)).replace('.',',')
record_tempo_monte_carlo = {num_ativos: tempo_elapsado}
records_tempo.append(record_tempo_monte_carlo)

dados_monte_carlo = {'Retorno': lista_retornos, 'Risco': lista_volatilidades}

for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
    dados_monte_carlo['peso '+ ticker] = [w[contador] for w in lista_pesos]

df_portfolios  = pd.DataFrame(dados_monte_carlo)

print(df_portfolios.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
df_portfolios.head(n = 10) # Visualizar os n primeiros portfólio dos portfolios criados com pesos aleatórios

In [ ]:
# Plot da Fronteira Eficiente
fig = px.scatter(df_portfolios, x='Risco', y='Retorno')
fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_pontos_fronteira.png"
    fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
portfolio_volatilidade_minima = df_portfolios.iloc[df_portfolios['Risco'].idxmin()] # idxmin() retorna o valor mínimo na coluna especificada
portfolio_volatilidade_minima

In [ ]:
portfolio_retorno_maximo = df_portfolios.iloc[df_portfolios['Retorno'].idxmax()] # idxmax() retorna o valor máximo na coluna especificada
portfolio_retorno_maximo

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: Poupança, acumulado 2021 ~ 2022 ~7,8997%

retorno_livre_risco_poupanca = 0.0790
portfolio_otimizado_poupanca = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_poupanca)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_poupanca

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: CDI/SELIC, acumulado 2022 ~12,3910%

retorno_livre_risco_cdi = 0.1239
portfolio_otimizado_cdi = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_cdi)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_cdi

In [ ]:
# Portfólio ótimo (melhor Sharpe Ratio)
# fator livre de risco definido como: FGTS (TR+3%), acumulado 2022 ~1,6314%

retorno_livre_risco_fgts = 0.0163
portfolio_otimizado_fgts = df_portfolios.iloc[((df_portfolios['Retorno']-retorno_livre_risco_fgts)/df_portfolios['Risco']).idxmax()]
portfolio_otimizado_fgts

In [ ]:
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
    go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
    go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
]

fig = go.Figure(data=[*objetos_scatter])
fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_pontos_portfolios_fronteira.png"
    # fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Fit simples para extrair a curva de fronteira eficiente
minimo_random_shape = 0.02
maximo_random_shape = 0.13

minimo_random_min_vol = 0.15
maximo_random_min_vol = 0.3

valor_aleatorio_1 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_1 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_1]['Risco']).idxmin()]

valor_aleatorio_2 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_2 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_2]['Risco']).idxmin()]

valor_aleatorio_3 = np.random.uniform(minimo_random_min_vol, maximo_random_min_vol)
ponto_aleatorio_fronteira_3 = df_portfolios.iloc[(df_portfolios.loc[df_portfolios['Retorno'] > valor_aleatorio_3]['Risco']).idxmin()]

valor_aleatorio_4 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_4 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_4)/df_portfolios['Risco']).idxmax()]

valor_aleatorio_5 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_5 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_5)/df_portfolios['Risco']).idxmax()]

valor_aleatorio_6 = np.random.uniform(minimo_random_shape, maximo_random_shape)
ponto_aleatorio_fronteira_6 = df_portfolios.iloc[((df_portfolios['Retorno']-valor_aleatorio_6)/df_portfolios['Risco']).idxmax()]

# Selecionando alguns pontos da fronteira
retornos_selecionados = np.array([portfolio_volatilidade_minima[1], ponto_aleatorio_fronteira_1[1], ponto_aleatorio_fronteira_2[1], ponto_aleatorio_fronteira_3[1], ponto_aleatorio_fronteira_4[1], ponto_aleatorio_fronteira_5[1], ponto_aleatorio_fronteira_6[1]])
riscos_selecionados = np.array([portfolio_volatilidade_minima[0], ponto_aleatorio_fronteira_1[0], ponto_aleatorio_fronteira_2[0], ponto_aleatorio_fronteira_3[0], ponto_aleatorio_fronteira_4[0], ponto_aleatorio_fronteira_5[0], ponto_aleatorio_fronteira_6[0]])

# Obs: Vamos fazer o fit polinomial com grau "g" de retornos em função de riscos, pois o contrário não é função
grau = 2
coeficientes = np.polyfit(riscos_selecionados, retornos_selecionados, grau)  # Retornos em função de riscos
modelo_ajustado = np.poly1d(coeficientes)

# Criando a curva de novos retornos e riscos após o fit e calibrando os limites
risco_novo = np.linspace(riscos_selecionados[0] - 0.05, riscos_selecionados[-1] + 0.02, 50)
retorno_novo = modelo_ajustado(risco_novo)

In [ ]:
# Plotando novamente com a fronteira eficiente explícita
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=retorno_novo, y=risco_novo, mode='lines', line=dict(color='black', width=3), name='Fronteira Eficiente'),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[ponto_aleatorio_fronteira_1.iloc[1]], y=[ponto_aleatorio_fronteira_1.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='diamond'),text=['ponto_aleatorio_fronteira_1'], name='ponto_aleatorio_fronteira_1'),
    go.Scatter(x=[ponto_aleatorio_fronteira_2.iloc[1]], y=[ponto_aleatorio_fronteira_2.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_2'], name='ponto_aleatorio_fronteira_2'),
    go.Scatter(x=[ponto_aleatorio_fronteira_3.iloc[1]], y=[ponto_aleatorio_fronteira_3.iloc[0]], mode='markers', marker=dict(color='brown', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_3'], name='ponto_aleatorio_fronteira_3'),
    go.Scatter(x=[ponto_aleatorio_fronteira_4.iloc[1]], y=[ponto_aleatorio_fronteira_4.iloc[0]], mode='markers', marker=dict(color='black', size=12, symbol='diamond'), text=['ponto_aleatorio_fronteira_4'], name='ponto_aleatorio_fronteira_4'),
    go.Scatter(x=[ponto_aleatorio_fronteira_5.iloc[1]], y=[ponto_aleatorio_fronteira_5.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='diamond'),  text=['ponto_aleatorio_fronteira_5'], name='ponto_aleatorio_fronteira_5'),
    go.Scatter(x=[ponto_aleatorio_fronteira_6.iloc[1]], y=[ponto_aleatorio_fronteira_6.iloc[0]], mode='markers', marker=dict(color='cyan', size=12, symbol='diamond'),  text=['ponto_aleatorio_fronteira_6'], name='ponto_aleatorio_fronteira_6'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")

if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_pontos_aleatorios.png"
    # fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

In [ ]:
# Plotando novamente com a fronteira eficiente explícita
objetos_scatter = [
    go.Scatter(x=df_portfolios['Risco'], y=df_portfolios['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
    go.Scatter(x=retorno_novo, y=risco_novo, mode='lines', line=dict(color='black', width=3), name='Fronteira Eficiente'),
    go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
    go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
    go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
    go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
    go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
]

fig = go.Figure(data=[*objetos_scatter])

fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_yaxes(tickformat=".1%")


if salvar_figuras:
    nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente.png"
    # fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

if mostrar_figuras:
    fig.show()

# Quantum

In [ ]:
# bounds = [(0,100) for i in range(num_qubits)]
# bounds

In [ ]:
otimizacao_portfolio = PortfolioOptimization(
    expected_returns = retorno_esperado_numpy, covariances = matrixz_cov_numpy,
    risk_factor = fator_risco, budget = limite_ativos_comprar#, bounds=bounds
)

programa_quadratico = otimizacao_portfolio.to_quadratic_program()
programa_quadratico_10 = programa_quadratico
programa_quadratico

### VQE

In [ ]:
ansatzes = [PauliTwoDesign(num_qubits, reps=1),
            EfficientSU2(num_qubits),
            RealAmplitudes(num_qubits),
            TwoLocal(num_qubits, "ry", "cz", reps=1, entanglement="full")]

lista_df_resultados_vqe = []
lista_parametros_iteracao_otimizador = []
lista_parametros_epoca_algoritmo_quantico = []

for indice_otimizador, otimizador in enumerate(otimizadores_VQE):

    otimizador_atual = type(otimizador).__name__

    lista_ansatz = []
    lista_otimizador = []

    lista_parametros = []
    lista_valor_energia = []
    lista_avaliacao_funcao_custo = []

    lista_epocas = []
    lista_composicao_otima = []
    lista_status_otimizacao = []
    lista_amostras_otimizacao = []
    lista_valor_otimo_encontrado = []
    lista_tempo_elapsado_otimizador = []
    lista_probabilidade_valor_otimo = []
    lista_quantidade_avaliacoes_total = []

    for indice_ansatz, ansatz in enumerate(ansatzes):
        ansatz_atual = ansatz.name

        print(f'\rOptimizer: {otimizador_atual}\t' + f'\tAnsatz: {ansatz_atual} \t\t\t\t\t\t', end='')

        melhores_parametros = None

        for epoca in range(quantidade_epocas_VQE):

            lista_avaliacoes_aux = []
            lista_parametros_aux = []
            lista_energia_aux = []

            def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
                lista_avaliacoes_aux.append(eval_count)
                lista_parametros_aux.append(parameters)
                lista_energia_aux.append(mean)
                clear_output(wait=False)

            # vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=otimizador, callback=guardar_parametros_chamada_custo)
            vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=otimizador, initial_point=melhores_parametros, callback=guardar_parametros_chamada_custo)
            vqe.random_seed = seed
            vqe_eigen = MinimumEigenOptimizer(vqe)
            resultado = vqe_eigen.solve(programa_quadratico)

            indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
            melhores_parametros = lista_parametros_aux[indice_menor_energia]
            # guardar_resultados_epoca_algoritmo_quantico_VQE(resultado, epoca)

            objeto_resultado = resultado.min_eigen_solver_result

            composicao_otima = resultado.x
            status_otimizacao = resultado.status
            amostras_otimizacao = resultado.samples
            valor_otimo_encontrado = objeto_resultado.optimal_value
            tempo_elapsado_otimizador = objeto_resultado.optimizer_time
            probabilidade_valor_otimo = objeto_resultado.best_measurement['probability']
            quantidade_avaliacoes_total = objeto_resultado.cost_function_evals

            lista_avaliacao_funcao_custo += lista_avaliacoes_aux
            lista_parametros += lista_parametros_aux
            lista_valor_energia += lista_energia_aux

            tamanho_listas = len(lista_energia_aux)

            lista_ansatz += [ansatz_atual for i in range(tamanho_listas)]
            lista_otimizador += [otimizador_atual for i in range(tamanho_listas)]

            lista_epocas += [epoca + 1 for i in range(tamanho_listas)]
            lista_composicao_otima += [composicao_otima for i in range(tamanho_listas)]
            lista_status_otimizacao += [status_otimizacao for i in range(tamanho_listas)]
            lista_amostras_otimizacao += [amostras_otimizacao for i in range(tamanho_listas)]
            lista_valor_otimo_encontrado += [valor_otimo_encontrado for i in range(tamanho_listas)]
            lista_tempo_elapsado_otimizador += [tempo_elapsado_otimizador for i in range(tamanho_listas)]
            lista_probabilidade_valor_otimo += [probabilidade_valor_otimo for i in range(tamanho_listas)]
            lista_quantidade_avaliacoes_total += [quantidade_avaliacoes_total for i in range(tamanho_listas)]
        
    dados_df_otimizador = {
        'parametros': lista_parametros,
        'epoca': lista_epocas,
        'avaliacao_funcao_custo': lista_avaliacao_funcao_custo,
        'valor_energia': lista_valor_energia,
        'ansatz': lista_ansatz,
        'otimizador': lista_otimizador,
        'composicao_otima': lista_composicao_otima,
        'status_otimizacao': lista_status_otimizacao,
        'amostras_otimizacao': lista_amostras_otimizacao,
        'quantidade_avaliacoes_total': lista_quantidade_avaliacoes_total,
        'tempo_elapsado_otimizador': lista_tempo_elapsado_otimizador,
        'valor_otimo_encontrado': lista_valor_otimo_encontrado,
        'probabilidade_valor_otimo': lista_probabilidade_valor_otimo,
    }

    df_otimizador = pd.DataFrame(dados_df_otimizador)

    lista_df_resultados_vqe.append(df_otimizador)

    array_avaliacao_funcao_custo_filtrado = np.empty([len(otimizadores_VQE)], dtype=object)
    array_valor_energia_filtrado = np.empty([len(otimizadores_VQE)], dtype=object)

    aux_contador_otimizador = 0

df_vqe = pd.concat(lista_df_resultados_vqe)
df_vqe = df_vqe.reset_index(drop=True)
df_vqe['nova_epoca'] = df_vqe['epoca'].diff().fillna(0).astype(bool)

clear_output(wait=True)
print(df_vqe.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_vqe

In [ ]:
# def checa_final_iteracao_otimizador(row):
#     for otimizador, ansatz, parametros in lista_parametros_iteracao_otimizador:
#         if row['otimizador'] == otimizador and row['ansatz'] == ansatz and tuple(row['parametros']) == tuple(parametros):
#             return True
#         elif row['otimizador'] == otimizador and row['ansatz'] == ansatz and row['avaliacao_funcao_custo'] == 1: # Lógica extra para a primeira chamada, pois o callback só da o retorno e nao o input.
#             return True
#     return False

# df_vqe['bool_fim_iteracao_otimizador'] = df_vqe.apply(checa_final_iteracao_otimizador, axis=1)
# df_vqe['iteracao_otimizador'] = np.nan

# # cria indice de iteracoes otimizador
# for _, df_agrupado in df_vqe.loc[df_vqe.bool_fim_iteracao_otimizador == True].groupby(['ansatz', 'otimizador'], as_index=False):
#     df_vqe.loc[df_agrupado.index, 'iteracao_otimizador'] = df_agrupado.reset_index().index + 1

# df_vqe

In [ ]:
with open(f'pickles/vqe_{num_ativos}_{ambiente_backend}.pickle', 'wb') as file:
    pickle.dump(df_vqe, file)

In [ ]:
# for otimizador in df_vqe['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe[df_vqe['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='avaliacao_funcao_custo')

#         x_data = df_ansatz['avaliacao_funcao_custo']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem)

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real = df_vqe_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_vqe_real, x='avaliacao_funcao_custo', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
# df_vqe_iteracao_otimizador = df_vqe.loc[df_vqe['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_vqe_iteracao_otimizador['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe_iteracao_otimizador[df_vqe_iteracao_otimizador['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='iteracao_otimizador')

#         x_data = df_ansatz['iteracao_otimizador']
#         y_data = df_ansatz['valor_energia']

#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real_iteracao_otimizador = df_vqe_real.loc[df_vqe_real['bool_fim_iteracao_otimizador'] == True]
# df_vqe_real_iteracao_otimizador = df_vqe_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_vqe_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_otimizador_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
df_vqe_epoca = df_vqe.loc[df_vqe['nova_epoca'] == True]

for otimizador in df_vqe_epoca['otimizador'].unique():
    df_otimizador = df_vqe_epoca[df_vqe_epoca['otimizador'] == otimizador]

    fig = go.Figure()

    for ansatz in df_otimizador['ansatz'].unique():
        df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
        df_ansatz = df_ansatz.sort_values(by='epoca')

        x_data = df_ansatz['epoca']
        y_data = df_ansatz['valor_energia']

        # Handling complex numbers by extracting real and imaginary parts
        y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_real,
            mode='lines',
            name=ansatz
        ))

    fig.update_layout(
        xaxis_title='Epoca',
        yaxis_title='Energia',
        title='',
        legend_title='ansatzs',
        width=width_padrao_display,
        height=height_padrao_display, 
        font=tamanho_padrao_fonte, 
        margin=tamanho_padrao_margem
    )

    if salvar_figuras:
        nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_vqe.png"
        fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

    if mostrar_figuras:
        fig.show()

In [ ]:
df_vqe_real = df_vqe.copy()
df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_vqe_real_epoca = df_vqe_real.loc[df_vqe_real['nova_epoca'] == True]
df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by='epoca')

fig = px.line(df_vqe_real_epoca, x='epoca', y='valor_energia', color='ansatz',
              facet_col='otimizador', facet_col_wrap=df_vqe_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='valor_otimo_encontrado', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='probabilidade_valor_otimo', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='tempo_elapsado_otimizador', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_tempo_elapsado_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='quantidade_avaliacoes_total', color='ansatz', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='ansatz', y='iteracao_otimizador', color='ansatz', facet_col='otimizador',
#              title='', 
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_otimizador_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### QAOA

In [ ]:
# Function to create a ring mixer
def create_ring_mixer(num_qubits):
    ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        ring_mixer *= Pauli(''.join(pauli_list))
    return ring_mixer

# Function to create a parity ring mixer
def create_parity_ring_mixer(num_qubits):
    parity_ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        parity_ring_mixer *= Pauli(''.join(pauli_list))
    pauli_list[-1] = 'X'  # Apply X gate on the last qubit
    parity_ring_mixer *= Pauli(''.join(pauli_list))
    return parity_ring_mixer

# Function to create a full mixer
def create_full_mixer(num_qubits):
    return Pauli('X'*num_qubits)

ring_mixer = create_ring_mixer(num_qubits)
parity_ring_mixer = create_parity_ring_mixer(num_qubits)
full_mixer = create_full_mixer(num_qubits)

In [ ]:
mixers = [('mixer_default', None),
          ('ring_mixer', ring_mixer),
          ('parity_ring_mixer', parity_ring_mixer),
          ('full_mixer', full_mixer)]

# print('\rOtimizacao completa\t\t\t\t\t\t\t')

lista_df_resultados_qaoa = []
lista_parametros_iteracao_otimizador = []
lista_parametros_epoca_algoritmo_quantico = []

for indice_otimizador, otimizador in enumerate(otimizadores_QAOA):

    otimizador_atual = type(otimizador).__name__

    lista_mixer = []
    lista_otimizador = []

    lista_parametros = []
    lista_valor_energia = []
    lista_avaliacao_funcao_custo = []

    lista_epocas = []
    lista_composicao_otima = []
    lista_status_otimizacao = []
    lista_amostras_otimizacao = []
    lista_valor_otimo_encontrado = []
    lista_tempo_elapsado_otimizador = []
    lista_probabilidade_valor_otimo = []
    lista_quantidade_avaliacoes_total = []

    for indice_mixer, tupla_mixer in enumerate(mixers):
        
        mixer = tupla_mixer[1]
        mixer_atual = tupla_mixer[0]

        print(f'\rOptimizer: {otimizador_atual}\t' + f'\tmixer: {mixer_atual} \t\t\t\t\t\t', end='')

        melhores_parametros = None

        for epoca in range(quantidade_epocas_QAOA):

            lista_avaliacoes_aux = []
            lista_parametros_aux = []
            lista_energia_aux = []

            def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
                lista_avaliacoes_aux.append(eval_count)
                lista_parametros_aux.append(parameters)
                lista_energia_aux.append(mean)
                clear_output(wait=False)

            # qaoa = QAOA(sampler=sampler, optimizer=otimizador, reps=3, mixer=mixer, callback=guardar_parametros_chamada_custo)
            qaoa = QAOA(sampler=sampler, optimizer=otimizador, reps=1, mixer=mixer, initial_point=melhores_parametros, callback=guardar_parametros_chamada_custo)
            qaoa_eigen = MinimumEigenOptimizer(qaoa)
            resultado = qaoa_eigen.solve(programa_quadratico)

            indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
            melhores_parametros = lista_parametros_aux[indice_menor_energia]

            objeto_resultado = resultado.min_eigen_solver_result

            composicao_otima = resultado.x
            status_otimizacao = resultado.status
            amostras_otimizacao = resultado.samples
            valor_otimo_encontrado = objeto_resultado.optimal_value
            tempo_elapsado_otimizador = objeto_resultado.optimizer_time
            probabilidade_valor_otimo = objeto_resultado.best_measurement['probability']
            quantidade_avaliacoes_total = objeto_resultado.cost_function_evals

            lista_avaliacao_funcao_custo += lista_avaliacoes_aux
            lista_parametros += lista_parametros_aux
            lista_valor_energia += lista_energia_aux

            tamanho_listas = len(lista_energia_aux)

            lista_mixer += [mixer_atual for i in range(tamanho_listas)]
            lista_otimizador += [otimizador_atual for i in range(tamanho_listas)]

            lista_epocas += [epoca + 1 for i in range(tamanho_listas)]
            lista_composicao_otima += [composicao_otima for i in range(tamanho_listas)]
            lista_status_otimizacao += [status_otimizacao for i in range(tamanho_listas)]
            lista_amostras_otimizacao += [amostras_otimizacao for i in range(tamanho_listas)]
            lista_valor_otimo_encontrado += [valor_otimo_encontrado for i in range(tamanho_listas)]
            lista_tempo_elapsado_otimizador += [tempo_elapsado_otimizador for i in range(tamanho_listas)]
            lista_probabilidade_valor_otimo += [probabilidade_valor_otimo for i in range(tamanho_listas)]
            lista_quantidade_avaliacoes_total += [quantidade_avaliacoes_total for i in range(tamanho_listas)]
        
    dados_df_otimizador = {
        'parametros': lista_parametros,
        'epoca': lista_epocas,
        'avaliacao_funcao_custo': lista_avaliacao_funcao_custo,
        'valor_energia': lista_valor_energia,
        'mixer': lista_mixer,
        'otimizador': lista_otimizador,
        'composicao_otima': lista_composicao_otima,
        'status_otimizacao': lista_status_otimizacao,
        'amostras_otimizacao': lista_amostras_otimizacao,
        'quantidade_avaliacoes_total': lista_quantidade_avaliacoes_total,
        'tempo_elapsado_otimizador': lista_tempo_elapsado_otimizador,
        'valor_otimo_encontrado': lista_valor_otimo_encontrado,
        'probabilidade_valor_otimo': lista_probabilidade_valor_otimo,
    }

    df_otimizador = pd.DataFrame(dados_df_otimizador)

    lista_df_resultados_qaoa.append(df_otimizador)

    array_avaliacao_funcao_custo_filtrado = np.empty([len(otimizadores_QAOA)], dtype=object)
    array_valor_energia_filtrado = np.empty([len(otimizadores_QAOA)], dtype=object)

    aux_contador_otimizador = 0

df_qaoa = pd.concat(lista_df_resultados_qaoa)
df_qaoa = df_qaoa.reset_index(drop=True)
df_qaoa['nova_epoca'] = df_qaoa['epoca'].diff().fillna(0).astype(bool)

clear_output(wait=True)
print(df_qaoa.head(10).to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

df_qaoa

In [ ]:
# def checa_final_iteracao_otimizador(row):
#     for otimizador, mixer, parametros in lista_parametros_iteracao_otimizador:
#         if row['otimizador'] == otimizador and row['mixer'] == mixer and tuple(row['parametros']) == tuple(parametros):
#             return True
#         elif row['otimizador'] == otimizador and row['mixer'] == mixer and row['avaliacao_funcao_custo'] == 1:
#             return True
#     return False

# df_qaoa['bool_fim_iteracao_otimizador'] = df_qaoa.apply(checa_final_iteracao_otimizador, axis=1)
# df_qaoa['iteracao_otimizador'] = np.nan

# # cria indice de iteracoes otimizador
# for _, df_agrupado in df_qaoa.loc[df_qaoa.bool_fim_iteracao_otimizador == True].groupby(['mixer', 'otimizador'], as_index=False):
#     df_qaoa.loc[df_agrupado.index, 'iteracao_otimizador'] = df_agrupado.reset_index().index + 1

# df_qaoa

In [ ]:
with open(f'pickles/qaoa_{num_ativos}_{ambiente_backend}.pickle', 'wb') as file:
    pickle.dump(df_qaoa, file)

In [ ]:
# for otimizador in df_qaoa['otimizador'].unique():
#     df_otimizador = df_qaoa[df_qaoa['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='avaliacao_funcao_custo')
        

#         x_data = df_mixer['avaliacao_funcao_custo']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )


#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real = df_qaoa_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_qaoa_real, x='avaliacao_funcao_custo', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')


# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
# df_qaoa_iteracao_otimizador = df_qaoa.loc[df_qaoa['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_qaoa_iteracao_otimizador['otimizador'].unique():
#     df_otimizador = df_qaoa_iteracao_otimizador[df_qaoa_iteracao_otimizador['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='iteracao_otimizador')

#         x_data = df_mixer['iteracao_otimizador']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [ ]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real_iteracao_otimizador = df_qaoa_real.loc[df_qaoa_real['bool_fim_iteracao_otimizador'] == True]
# df_qaoa_real_iteracao_otimizador = df_qaoa_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_qaoa_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_otimizador_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [ ]:
df_qaoa_epoca = df_qaoa.loc[df_qaoa['nova_epoca'] == True]

for otimizador in df_qaoa_epoca['otimizador'].unique():
    df_otimizador = df_qaoa_epoca[df_qaoa_epoca['otimizador'] == otimizador]

    fig = go.Figure()

    for mixer in df_otimizador['mixer'].unique():
        df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
        df_mixer = df_mixer.sort_values(by='epoca')

        x_data = df_mixer['epoca']
        y_data = df_mixer['valor_energia']

        # Handling complex numbers by extracting real and imaginary parts
        y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

        fig.add_trace(go.Scatter(
            x=x_data,
            y=y_real,
            mode='lines',
            name=mixer
        ))

    fig.update_layout(
        xaxis_title='Epoca',
        yaxis_title='Energia',
        title='',
        legend_title='mixers',
        width=width_padrao_display,
        height=height_padrao_display, 
        font=tamanho_padrao_fonte, 
        margin=tamanho_padrao_margem
    )

    if salvar_figuras:
        nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_qaoa.png"
        fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

    if mostrar_figuras:
        fig.show()

In [ ]:
df_qaoa_real = df_qaoa.copy()
df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_qaoa_real_epoca = df_qaoa_real.loc[df_qaoa_real['nova_epoca'] == True]
df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by='epoca')

fig = px.line(df_qaoa_real_epoca, x='epoca', y='valor_energia', color='mixer',
              facet_col='otimizador', facet_col_wrap=df_qaoa_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='valor_otimo_encontrado', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_valor_otimo_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='probabilidade_valor_otimo', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_qaoa.png" 
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='tempo_elapsado_otimizador', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_tempo_elapsado_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='quantidade_avaliacoes_total', color='mixer', facet_col='otimizador',
             title='',
             width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [ ]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='mixer', y='iteracao_otimizador', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_otimizador_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### Plots finais nas fronteiras eficientes

In [ ]:
# lista_pesos_portfolio_quantico = []
# composicao_portfolio_quantico = df_qaoa.composicao_otima.iloc[0]

# pesos_portfolio_quantico = composicao_portfolio_quantico/np.sum(composicao_portfolio_quantico)
# lista_pesos_portfolio_quantico.append(pesos_portfolio_quantico)

# retorno_esperado_portfolio_quantico = np.dot(pesos_portfolio_quantico, retorno_esperado)

# variancia_portfolio_quantico = matrixz_cov.mul(pesos_portfolio_quantico, axis=0).mul(pesos_portfolio_quantico, axis=1).sum().sum()
# desvio_padrao_diario_portfolio_quantico = np.sqrt(variancia_portfolio_quantico)
# desvio_padrao_anualizado_portfolio_quantico = desvio_padrao_diario_portfolio_quantico*np.sqrt(252)

# dados_portfolio_quantico = {'Retorno': retorno_esperado_portfolio_quantico, 'Risco': desvio_padrao_anualizado_portfolio_quantico}

# for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
#     dados_portfolio_quantico['peso '+ ticker] = [w[contador] for w in lista_pesos_portfolio_quantico]

# df_portfolio_quantico  = pd.DataFrame(dados_portfolio_quantico)

# print('Carteira otimizada com algoritmos quânticos:')
# print(df_portfolio_quantico.to_latex(index=False,
#                   formatters={"name": str.upper},
#                   float_format="{:.1f}".format,
# ))
# display(df_portfolio_quantico)

# sem_restricao_lista_volatilidades = []
# sem_restricao_lista_retornos = []
# sem_restricao_lista_pesos = []
# sem_restricao_dados_monte_carlo = {}

# for portfolio in range(num_portfolios/3):

#     pesos_portfolio = np.random.random(num_ativos)
#     pesos_portfolio = pesos_portfolio/np.sum(pesos_portfolio)
    
#     sem_restricao_lista_pesos.append(pesos_portfolio)
#     retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
#     sem_restricao_lista_retornos.append(retornos_portfolio)

#     variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
#     desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
#     desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

#     sem_restricao_lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

# sem_restricao_dados_monte_carlo = {'Retorno': sem_restricao_lista_retornos, 'Risco': sem_restricao_lista_volatilidades}

# for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
#     sem_restricao_dados_monte_carlo['peso '+ ticker] = [w[contador] for w in sem_restricao_lista_pesos]

# df_portfolios_sem_restricao = pd.DataFrame(sem_restricao_dados_monte_carlo)

# print('Carteiras sem restrição:')
# print(df_portfolios_sem_restricao.head(5).to_latex(index=False,
#                   formatters={"name": str.upper},
#                   float_format="{:.1f}".format,
# ))
# display(df_portfolios_sem_restricao.head(n = 3))

# cardinalidade_lista_volatilidades = []
# cardinalidade_lista_retornos = []
# cardinalidade_lista_pesos = []
# cardinalidade_dados_monte_carlo = {}

# for portfolio in range(num_portfolios/2):
#     escolha_ativos_portfolio = np.random.random(num_ativos)
#     pesos_ativos = np.random.dirichlet(np.ones(limite_ativos_comprar))
#     pesos_ativos *= 100

#     indices_ativos_escolhidos = np.argsort(escolha_ativos_portfolio)[-limite_ativos_comprar:]

#     escolha_ativos_portfolio[:] = 0
#     escolha_ativos_portfolio[indices_ativos_escolhidos] = pesos_ativos

#     pesos_portfolio = escolha_ativos_portfolio/np.sum(escolha_ativos_portfolio)

#     cardinalidade_lista_pesos.append(pesos_portfolio)

#     retornos_portfolio = np.dot(pesos_portfolio, retorno_esperado)
#     cardinalidade_lista_retornos.append(retornos_portfolio)

#     variancia_portfolio = matrixz_cov.mul(pesos_portfolio, axis=0).mul(pesos_portfolio, axis=1).sum().sum()
#     desvio_padrao_diario_portfolio = np.sqrt(variancia_portfolio)
#     desvio_padrao_anualizado_portfolio = desvio_padrao_diario_portfolio*np.sqrt(252)

#     cardinalidade_lista_volatilidades.append(desvio_padrao_anualizado_portfolio)

# cardinalidade_dados_monte_carlo = {'Retorno': cardinalidade_lista_retornos, 'Risco': cardinalidade_lista_volatilidades}

# for contador, ticker in enumerate(df_fechamentos.columns.tolist()):
#     cardinalidade_dados_monte_carlo['peso '+ ticker] = [w[contador] for w in cardinalidade_lista_pesos]

# df_portfolios_cardinalidade  = pd.DataFrame(cardinalidade_dados_monte_carlo)

# print('Carteiras com restrição de cardinalidade:')
# print(df_portfolios_cardinalidade.head(5).to_latex(index=False,
#                   formatters={"name": str.upper},
#                   float_format="{:.1f}".format,
# ))
# display(df_portfolios_cardinalidade.head(n = 3))


# objetos_scatter = [
#     go.Scatter(x=df_portfolios_sem_restricao['Risco'], y=df_portfolios_sem_restricao['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
#     go.Scatter(x=df_portfolios_cardinalidade['Risco'], y=df_portfolios_cardinalidade['Retorno'], mode='markers', marker=dict(color='cyan'), text=['Retorno Máximo'], showlegend=False),
#     go.Scatter(x=[portfolio_retorno_maximo.iloc[1]], y=[portfolio_retorno_maximo.iloc[0]], mode='markers', marker=dict(color='red', size=12, symbol='circle'),text=['Retorno Máximo'], name='Retorno Máximo'),
#     go.Scatter(x=[portfolio_volatilidade_minima.iloc[1]], y=[portfolio_volatilidade_minima.iloc[0]], mode='markers', marker=dict(color='blue', size=12, symbol='star'),text=['Risco Mínimo'], name='Risco Mínimo'),
#     go.Scatter(x=[portfolio_otimizado_poupanca.iloc[1]], y=[portfolio_otimizado_poupanca.iloc[0]], mode='markers', marker=dict(color='magenta', size=12, symbol='square'),text=['Sharpe (Poupança)'], name='Sharpe (Poupança)'),
#     go.Scatter(x=[portfolio_otimizado_cdi.iloc[1]], y=[portfolio_otimizado_cdi.iloc[0]], mode='markers', marker=dict(color='green', size=12, symbol='diamond'),text=['Sharpe (CDI)'], name='Sharpe (CDI)'),
#     go.Scatter(x=[portfolio_otimizado_fgts.iloc[1]], y=[portfolio_otimizado_fgts.iloc[0]], mode='markers', marker=dict(color='yellow', size=12, symbol='cross'),text=['Sharpe (FGTS)'], name='Sharpe (FGTS)'),
#     go.Scatter(x=[df_portfolio_quantico['Risco'].iloc[0]], y=[df_portfolio_quantico['Retorno'].iloc[0]], mode='markers', marker=dict(color='black', size=20, symbol='x'),text=['Portfólio Otimizado Quântico'], name='Portfólio Otimizado Quântico'),
# ]

# fig = go.Figure(data=[*objetos_scatter])

# fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_yaxes(tickformat=".1%")


# if salvar_figuras:
#     nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_quantico_comparativo.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

# if mostrar_figuras:
#     fig.show()


# objetos_scatter = [
#     go.Scatter(x=df_portfolios_sem_restricao['Risco'], y=df_portfolios_sem_restricao['Retorno'], mode='markers', text=['Retorno Máximo'], showlegend=False),
#     go.Scatter(x=df_portfolios_cardinalidade['Risco'], y=df_portfolios_cardinalidade['Retorno'], mode='markers', marker=dict(color='cyan'), text=['Retorno Máximo'], showlegend=False),
#     go.Scatter(x=[df_portfolio_quantico['Risco'].iloc[0]], y=[df_portfolio_quantico['Retorno'].iloc[0]], mode='markers', marker=dict(color='black', size=20, symbol='x'),text=['Portfólio Otimizado Quântico'], name='Portfólio Otimizado Quântico'),
# ]

# fig = go.Figure(data=[*objetos_scatter])

# fig.update_layout(xaxis_title='Risco', yaxis_title='Retorno', title='', width=width_padrao_display, height=height_padrao_display, font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_yaxes(tickformat=".1%")


# if salvar_figuras:
#     nome_figura = f"figuras_dados/{num_ativos}_fronteira_eficiente_quantico.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

# if mostrar_figuras:
#     fig.show()

# Tabela Versionamento

In [ ]:
import qiskit.tools.jupyter
import warnings

warnings.filterwarnings("ignore")

%qiskit_version_table